# Noun Phrase Chunking

In [1]:
import pandas as pd

import spacy

import nltk
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag

In [2]:
raw = "A chatbot (also known as a talkbot, chatterbot, Bot, IM bot, interactive agent, or Artificial Conversational Entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods. Such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the Turing test. Chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition. Some chatterbots use sophisticated natural language processing systems, but many simpler systems scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database."

## Chunking with NLTK

The method for chunking text with NLTK is slightly different than with Spacy.  It's more complicated, but ultimatley more flexible.  It uses the combination of POS tagging and `RegEx` to parse groups of words for patterns.  Because of the use of `RegEx`, it's possible to create any sets of POS tags.

In [3]:
def reg_chunker(sent, expression):

    sent = pos_tag(word_tokenize(sent))
    cp = nltk.RegexpParser(expression)
    chunked = cp.parse(sent)

    for chunk in chunked.subtrees(filter=lambda t: t.label() == 'NP'):
        print(chunk)

In [4]:
# Chunk 1: Noun followed by Noun
reg_chunker(raw, r'NP: {<NN.?>+<NN.?>}')

(NP IM/NNP bot/NN)
(NP Artificial/NNP Conversational/NNP Entity/NNP)
(NP computer/NN program/NN)
(NP Turing/NNP test/NN)
(NP dialog/NN systems/NNS)
(NP customer/NN service/NN)
(NP information/NN acquisition/NN)
(NP language/NN processing/NN systems/NNS)
(NP simpler/NN systems/NNS)
(NP wording/NN pattern/NN)


In [5]:
# Chunk 2: Adjective Follwed by Singular Noun
reg_chunker(raw, r'NP: {<JJ>+<NN>}')

(NP interactive/JJ agent/NN)
(NP artificial/JJ intelligence/NN)
(NP conversational/JJ partner/NN)
(NP sophisticated/JJ natural/JJ language/NN)
(NP many/JJ simpler/NN)
(NP similar/JJ wording/NN)


## Chunking with Spacy

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for chunk in doc.noun_chunks:
    print(chunk.text)

Autonomous cars
insurance liability
manufacturers


In [8]:
doc = nlp(raw)
for chunk in doc.noun_chunks:
    print(chunk.text)

A chatbot
a talkbot
chatterbot
Bot
IM bot
interactive agent
Artificial Conversational Entity
a computer program
an artificial intelligence
which
a conversation
auditory or textual methods
Such programs
a human
a conversational partner
the Turing test
Chatbots
dialog systems
various practical purposes
customer service
information acquisition
Some chatterbots
sophisticated natural language processing systems
many simpler systems
keywords
the input
a reply
the most matching keywords
the most similar wording pattern
a database


## Chunking Clothing Reviews with Spacy

In [9]:
df = pd.read_csv("ClothingReviews.csv")
df.head()

,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Department Name,Class Name
0,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Intimate,Intimates
1,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,Dresses,Dresses
2,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,Dresses,Dresses
3,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,Bottoms,Pants
4,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,Tops,Blouses


In [10]:
df.dropna(subset=['Review Text'], inplace=True)

In [11]:
def np_tag(text):
    
    df = pd.DataFrame(columns = ['CHUNK'])
    
    doc = nlp(text)
    for chunk in doc.noun_chunks:
        df = df.append({'CHUNK': chunk.text}, ignore_index=True)
        
    return df

In [12]:
# Convert our reviews to lowercase to simplify our search
df["Review Text"] = df["Review Text"].str.lower()

# Find only reviews that have the word 'dress' in them
filter = df['Review Text'].str.contains('top')
df_dress = df[filter].copy()
df_dress.shape

(5884, 8)

In [13]:
# Create an empty dataframe to store the results
df_np = pd.DataFrame(columns = ['CHUNK'])

# Iterate through the reviews and extra non-phrases for the reivews with "small or little"
df_np = np_tag(df_dress['Review Text'].to_string())
df_np.shape

(17926, 1)

In [14]:
# Show the top 10 noun phrases - notice that there are a lot of filler words (stop words)
df_np.groupby('CHUNK')['CHUNK'].count().\
    reset_index(name='count').sort_values(['count'],ascending=False).head(10)

,CHUNK,count
1954,i,4355
4048,this top,1775
1975,it,1323
3867,this,898
3928,this dress,378
2079,love,126
2116,me,125
2877,that,121
4014,this shirt,102
3735,the top,99


In [15]:
# As opposed to removing stop words, we can filter out rows in the dataframe
# that have the stop words.  This is a better way for noun phrases since we won't lose
# the context of the phrases during our prior extraction. 
filter = (df_np['CHUNK'].str.contains('this')) | \
         (df_np['CHUNK'].str.contains('the')) | \
         (df_np['CHUNK'].str.contains('that')) | \
         (df_np['CHUNK'].str.contains('my')) | \
         (df_np['CHUNK'].str.contains('a'))
df_np = df_np[-filter]

In [16]:
# Filter for words with spaces, so that we get only phrases with more than one word.
filter = (df_np['CHUNK'].str.contains(' '))
df_np = df_np[filter]

In [17]:
df_np.groupby('CHUNK')['CHUNK'].count().\
    reset_index(name='count').sort_values(['count'],ascending=False).head(30)

,CHUNK,count
244,both colors,17
319,high hopes,14
47,love,11
161,2 colors,8
25,cute top,6
503,very cute top,6
488,two colors,5
455,such high hopes,5
513,very pretty top,3
39,very cute top,3
